# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT

import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

## 필요없는 feature 제거

In [3]:
data.drop(['PassengerId', 'Cabin', 'Combi', 'Name',], axis=1, inplace=True)

## 결측값 제거

In [4]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## 원핫 인코딩

In [5]:
data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
data['Cabin3'] = data['Cabin3'].astype(bool)

In [6]:
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)

## 원핫 인코딩

In [7]:
col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [8]:
data_scaled(data, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,PSO J318.5-22,TRAPPIST-1e,A,B,C,D,E,F,G,T
0,False,-1.170228,True,0.712274,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,...,0,1,0,1,0,0,0,0,0,0
1,False,-1.170228,False,-0.332624,False,-0.168530,-0.275148,-0.241196,0.220460,-0.224520,...,0,1,0,0,0,0,0,1,0,0
2,False,-1.170228,False,2.035811,True,-0.268567,1.959032,-0.282832,5.745469,-0.220106,...,0,1,1,0,0,0,0,0,0,0
3,False,-1.170228,False,0.294315,False,-0.333743,0.522818,0.335048,2.711463,-0.092988,...,0,1,1,0,0,0,0,0,0,0
4,False,-1.168274,False,-0.889902,False,0.125518,-0.236941,-0.031350,0.234796,-0.261596,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,-0.978724,True,0.851594,True,-0.333743,3.990274,-0.282832,1.200732,-0.198037,...,0,0,1,0,0,0,0,0,0,0
8689,True,1.758999,False,-0.750583,False,-0.333743,-0.280785,-0.282832,-0.271469,-0.263361,...,1,0,0,0,0,0,0,0,1,0
8690,False,1.760953,False,-0.193304,False,-0.333743,-0.280785,2.834877,-0.270573,-0.263361,...,0,1,0,0,0,0,0,0,1,0
8691,False,0.017878,False,0.224655,False,-0.333743,0.376253,-0.282832,0.044835,2.592370,...,0,0,0,0,0,0,1,0,0,0


In [9]:
data.columns

Index(['CryoSleep', 'Cabin2', 'Cabin3', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Earth',
       'Europa', 'Mars', '55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', 'A',
       'B', 'C', 'D', 'E', 'F', 'G', 'T'],
      dtype='object')

In [10]:
df = data.iloc[:,:]

In [11]:
df['Transported'] = df.Transported.replace({True:1, False:0})

# 데이터셋 분리

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Transported'], axis=1),
                                                    df.Transported, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [13]:
X_train.shape

(4831, 24)

In [14]:
y_train.shape

(4831,)

# XGBoost

## HyperOPT

In [15]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [16]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100,max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [17]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.90942	validation_1-auc:0.85396                                                                  
[1]	validation_0-auc:0.92857	validation_1-auc:0.86344                                                                  
[2]	validation_0-auc:0.93736	validation_1-auc:0.86882                                                                  
[3]	validation_0-auc:0.94227	validation_1-auc:0.87816                                                                  
[4]	validation_0-auc:0.94791	validation_1-auc:0.87718                                                                  
[5]	validation_0-auc:0.94975	validation_1-auc:0.87916                                                                  
[6]	validation_0-auc:0.95155	validation_1-auc:0.88135                                                                  
[7]	validation_0-auc:0.95402	validation_1-auc:0.88157                                                                  
[8]	validation_0-auc:0.95391	validation_

[18]	validation_0-auc:0.96540	validation_1-auc:0.88660                                                                 
[19]	validation_0-auc:0.96666	validation_1-auc:0.88742                                                                 
[20]	validation_0-auc:0.96666	validation_1-auc:0.88742                                                                 
[21]	validation_0-auc:0.96666	validation_1-auc:0.88742                                                                 
[22]	validation_0-auc:0.96666	validation_1-auc:0.88742                                                                 
[23]	validation_0-auc:0.96666	validation_1-auc:0.88742                                                                 
[24]	validation_0-auc:0.96700	validation_1-auc:0.88693                                                                 
[25]	validation_0-auc:0.96700	validation_1-auc:0.88693                                                                 
[26]	validation_0-auc:0.96700	validation

[36]	validation_0-auc:0.96894	validation_1-auc:0.88972                                                                 
[37]	validation_0-auc:0.96894	validation_1-auc:0.88972                                                                 
[38]	validation_0-auc:0.96927	validation_1-auc:0.88972                                                                 
[39]	validation_0-auc:0.96969	validation_1-auc:0.89049                                                                 
[40]	validation_0-auc:0.96979	validation_1-auc:0.89068                                                                 
[41]	validation_0-auc:0.96979	validation_1-auc:0.89068                                                                 
[42]	validation_0-auc:0.96979	validation_1-auc:0.89068                                                                 
[43]	validation_0-auc:0.96979	validation_1-auc:0.89068                                                                 
[0]	validation_0-auc:0.87261	validation_

[60]	validation_0-auc:0.94343	validation_1-auc:0.89377                                                                 
[61]	validation_0-auc:0.94340	validation_1-auc:0.89389                                                                 
[62]	validation_0-auc:0.94366	validation_1-auc:0.89399                                                                 
[63]	validation_0-auc:0.94366	validation_1-auc:0.89399                                                                 
[64]	validation_0-auc:0.94410	validation_1-auc:0.89428                                                                 
[65]	validation_0-auc:0.94411	validation_1-auc:0.89418                                                                 
[66]	validation_0-auc:0.94411	validation_1-auc:0.89418                                                                 
[67]	validation_0-auc:0.94427	validation_1-auc:0.89431                                                                 
[68]	validation_0-auc:0.94433	validation

[28]	validation_0-auc:0.93364	validation_1-auc:0.88553                                                                 
[29]	validation_0-auc:0.93394	validation_1-auc:0.88532                                                                 
[30]	validation_0-auc:0.93417	validation_1-auc:0.88526                                                                 
[31]	validation_0-auc:0.93453	validation_1-auc:0.88504                                                                 
[32]	validation_0-auc:0.93471	validation_1-auc:0.88567                                                                 
[33]	validation_0-auc:0.93491	validation_1-auc:0.88554                                                                 
[34]	validation_0-auc:0.93488	validation_1-auc:0.88558                                                                 
[35]	validation_0-auc:0.93502	validation_1-auc:0.88595                                                                 
[36]	validation_0-auc:0.93620	validation

...

[12]	validation_0-auc:0.93073	validation_1-auc:0.88911                                                                 
[13]	validation_0-auc:0.93206	validation_1-auc:0.88824                                                                 
[14]	validation_0-auc:0.93240	validation_1-auc:0.88877                                                                 
[15]	validation_0-auc:0.93515	validation_1-auc:0.88958                                                                 
[16]	validation_0-auc:0.93620	validation_1-auc:0.88986                                                                 
[17]	validation_0-auc:0.93723	validation_1-auc:0.89084                                                                 
[18]	validation_0-auc:0.93811	validation_1-auc:0.89097                                                                 
[19]	validation_0-auc:0.93975	validation_1-auc:0.89026                                                                 
[20]	validation_0-auc:0.94120	validation

[32]	validation_0-auc:0.94663	validation_1-auc:0.89496                                                                 
[33]	validation_0-auc:0.94719	validation_1-auc:0.89519                                                                 
[34]	validation_0-auc:0.94789	validation_1-auc:0.89488                                                                 
[35]	validation_0-auc:0.94832	validation_1-auc:0.89460                                                                 
[36]	validation_0-auc:0.94860	validation_1-auc:0.89475                                                                 
[37]	validation_0-auc:0.94891	validation_1-auc:0.89399                                                                 
[38]	validation_0-auc:0.94974	validation_1-auc:0.89379                                                                 
[39]	validation_0-auc:0.95051	validation_1-auc:0.89334                                                                 
[40]	validation_0-auc:0.95051	validation

[38]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[39]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[40]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[41]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[42]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[43]	validation_0-auc:0.95227	validation_1-auc:0.89505                                                                 
[44]	validation_0-auc:0.95244	validation_1-auc:0.89531                                                                 
[45]	validation_0-auc:0.95244	validation_1-auc:0.89531                                                                 
[46]	validation_0-auc:0.95244	validation

[20]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[21]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[22]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[23]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[24]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[25]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[26]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[27]	validation_0-auc:0.94495	validation_1-auc:0.88889                                                                 
[28]	validation_0-auc:0.94643	validation

[23]	validation_0-auc:0.94578	validation_1-auc:0.89698                                                                 
[24]	validation_0-auc:0.94578	validation_1-auc:0.89698                                                                 
[25]	validation_0-auc:0.94641	validation_1-auc:0.89609                                                                 
[26]	validation_0-auc:0.94656	validation_1-auc:0.89620                                                                 
[27]	validation_0-auc:0.94940	validation_1-auc:0.89594                                                                 
[28]	validation_0-auc:0.94999	validation_1-auc:0.89582                                                                 
[29]	validation_0-auc:0.94999	validation_1-auc:0.89582                                                                 
[30]	validation_0-auc:0.95016	validation_1-auc:0.89570                                                                 
[31]	validation_0-auc:0.95117	validation

## 모델 학습

In [18]:
xgbo = xgb.XGBClassifier(colsample_bytree=best['colsample_bytree'],
                         gamma=best['gamma'], max_depth=int(best['max_depth']),
                         learning_rate=best['learning_rate'],
                         min_child_weight=int(best['min_child_weight']),
                         n_estimators=100,
                         random_state=109)
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7453678484017885, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.0022199018584637, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.09109346300898982, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=7, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

## 평가 지표

In [19]:
# 평가용 함수
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [20]:
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

### 훈련셋

In [21]:
get_clf_eval(y_train, train_pred, train_proba)

오차 행렬
[[2044  352]
 [ 318 2117]]
정확도: 0.8613, 정밀도: 0.8574,     재현율: 0.8694, F1: 0.8634


### 테스트셋

In [22]:
get_clf_eval(y_test, test_pred, test_proba)

오차 행렬
[[845 222]
 [214 867]]
정확도: 0.7970, 정밀도: 0.7961,     재현율: 0.8020, F1: 0.7991


### 검증셋

In [23]:
get_clf_eval(y_val, val_pred, val_proba)

오차 행렬
[[644 150]
 [141 676]]
정확도: 0.8194, 정밀도: 0.8184,     재현율: 0.8274, F1: 0.8229


### feature 중요도

In [24]:
xgbo.feature_importances_.shape

(24,)

In [25]:
fi = pd.DataFrame(xgbo.feature_importances_, index=X_train.columns)

In [26]:
fi

,0
CryoSleep,0.395417
Cabin2,0.016745
Cabin3,0.021160
Age,0.014614
VIP,0.004827
RoomService,0.048333
FoodCourt,0.037603
ShoppingMall,0.024292
Spa,0.039598
VRDeck,0.043584
